### INTEGRACIÓN


#### Descripción del trabajo:

    Este trabajo simula una parte del funcionamiento de una red de blockchain. El archivo Master lee el último bloque de la cadena que está guardado en un archivo "last_block.txt". Después, se pregunta al usuario cuántos workers desea crear para procesar el siguiente bloque de la cadena. Se envía ese dato junto con el último bloque mediante TCP al módulo Worker, que se encarga de repartir la tarea de procesamiento entre el número de workers establecido por el usuario. 
    
    La tarea del módulo Worker consiste en encontrar un número entero (llamada "nonce") que, si se añade al último bloque de la cadena, genera un hash que empieza por 6 ceros. Esta tarea es tediosa y poco eficiente, pues se deben recorrer todos los números enteros uno a uno hasta encontrar el que genera el hash deseado. Por ello, es buena idea distribuirla entre diferentes workers, asignando a cada uno de ellos un intervalo de menor tamaño en el que deben buscar ese número. Como se podrá observar, el tiempo de procesamiento es mucho mayor con un worker que con varios, lo que demuestra la utilidad de la distribución. Eso sí, un número de workers superior a 8 dejará de ser eficiente debido a las limitaciones de la CPU.
    
   
#### Instrucciones:

    Para correr el proyecto, primero debe ejecutarse el módulo Worker. Una vez esté corriendo, se ejecuta el módulo Master y se introduce el número de workers deseado. En ese momento, el módulo Worker lanza el número de threads correspondientes, y empieza a mostrar por pantalla el progreso de la tarea. Cuando un worker encuentra un número que genera un Hash con 5 ceros (en vez de 6), este se imprime. Cuando un worker encuentra el número que genera un hash con 6 ceros (el Nonce), se paran automáticamente todos los workers y se envía el resultado de vuelta al módulo Master. Entonces, en este módulo se muestra el Nonce encontrado, el worker que lo ha descubierto, así como el tiempo de procesamiento total empleado. Finalmente, se crea un nuevo bloque a partir del Nonce encontrado, y se guarda en el archivo "last_block.txt".
    
    El módulo Worker no se para tras encontrar un Nonce, sino que se queda esperando a una nueva petición del Master. Si se desea parar este módulo, debe introducirse 0 como número de workers. En cambio, si se desea continuar generando bloques, simplemente hay que volver a ejecutar el módulo Master.

In [5]:
import socket
import sys

block = ""

try:
    with open("last_block.txt") as f:
        block = f.read()
        print("Last block of chain: ", block)
                
except:
    block="c29e1bd4aeb7c4504c1b3d993bc17a13"
    print("Error file not found")
        
workers = input("Desired number of workers: ")


# Create a TCP/IP socket
connection = False

try:
    server_address = ("localhost", 6780)
    sock = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)
    sock.connect(server_address)
    sock.sendall((block + " " + str(workers)).encode())
    connection = True

except:
    print("Error connecting to the server")

stop = False
if connection == True:
    while stop == False:
        msg = sock.recv(1024).decode()
        data = msg.split()
        if data[0] == "success":
            worker = data[1]
            nonce = data[2]
            time = data[3]
            print("Worker " + worker + " found Nonce: " + nonce + "   Time elapsed: " + time)
            stop = True
            f = open("last_block.txt", "w+")
            new_block = block + str(int(nonce)-1)
            f.write(new_block)
            f.close()
            print("New block created: " +new_block )
            
        elif data[0] == "Shutting":
            print(msg)
            stop = True
        else:
            print(msg)



Last block of chain:  c29e1bd4aeb7c4504c1b3d993bc17a13411891709338663
Desired number of workers: 2
Worker 0 found Nonce: 13969957   Time elapsed: 35.26750564575195
New block created: c29e1bd4aeb7c4504c1b3d993bc17a1341189170933866313969956
